In [ ]:
import requests
import pandas as pd
import joblib
import time

# Load the trained model and scaler
model = joblib.load(r'C:\Users\JANU\nsvmrbf.pkl')  # Path to your model file
scaler = joblib.load(r'C:\Users\JANU\nscal.pkl')  # Path to your scaler file

# Map the prediction values to activities
prediction_mapping = {
    1: "Fall",
    3: "Jog",
    4: "Walk",
    5: "Std",
    6: "Run",
    7: "Jump"
}

# Fetch data from Firebase Realtime Database
def fetch_data_from_firebase(firebase_url, data_path):
    try:
        response = requests.get(firebase_url + data_path + '.json')
        response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx
        data = response.json()
        return data
    except requests.RequestException as e:
        print(f"Error fetching data from Firebase: {e}")
        return {}

def preprocess_and_predict(data):
    try:
        # Ensure all necessary features are present
        required_features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']
        if not all(feature in data for feature in required_features):
            print("Missing one or more required features.")
            return None

        # Remove 'Label' field if present, since it's not required for prediction
        if 'Label' in data:
            del data['Label']

        # Convert values to float
        data = {k: float(v) for k, v in data.items()}

        # Convert data to DataFrame
        df = pd.DataFrame([data])  # Wrap data in a list to handle single entries

        # Select only the features that the model was trained on
        df = df[required_features]

        # Scale the features
        X = scaler.transform(df)

        # Make prediction
        predictions = model.predict(X)

        return int(predictions[0])  # Convert prediction to standard Python int

    except Exception as e:
        print(f"Error during preprocessing or prediction: {e}")
        return None

# Send predictions to Firebase
def send_predictions_to_firebase(activity, prediction_value, firebase_url, result_path):
    try:
        # Create a dictionary with activity and prediction value
        result_data = {
            'activity': activity,          # The corresponding activity name (Fall, Jog, etc.)
            'prediction': prediction_value  # The raw prediction value
        }
        
        # Send the data to Firebase
        response = requests.put(firebase_url + result_path + '.json', json=result_data)
        response.raise_for_status()
        print("Prediction successfully sent to Firebase")
    except requests.RequestException as e:
        print(f"Error sending activity to Firebase: {e}")

# Main loop
firebase_url = "https://harsha-3a181-default-rtdb.firebaseio.com/"  # Your Firebase Realtime Database URL
data_path = "/Falldetect"  # Adjust the path to match where your data is stored in Firebase
result_path = "/predictions"  # Path to send activity to Firebase

while True:  # Infinite loop
    print("Fetching new data...")
    new_data = fetch_data_from_firebase(firebase_url, data_path)
    if new_data:
        print("Data received:", new_data)
        prediction = preprocess_and_predict(new_data)
        if prediction is not None:
            # Map the prediction to the corresponding activity
            activity = prediction_mapping.get(prediction, "Unknown Activity")
            print(f"Prediction made: {activity} (Value: {prediction})")
            
            # Send both the activity name and the prediction value to Firebase
            send_predictions_to_firebase(activity, prediction, firebase_url, result_path)
    else:
        print("No new data found.")
    time.sleep(10)  # Wait for 10 seconds before the next iteration


Fetching new data...
Data received: {'AccelX': '-1.51', 'AccelY': '0.12', 'AccelZ': '9.03', 'GyroX': '-3.00', 'GyroY': '2.75', 'GyroZ': '1.96'}
Prediction made: Fall (Value: 1)
Prediction successfully sent to Firebase
Fetching new data...
Data received: {'AccelX': '-1.51', 'AccelY': '0.12', 'AccelZ': '9.03', 'GyroX': '-3.00', 'GyroY': '2.75', 'GyroZ': '1.96'}
Prediction made: Fall (Value: 1)
Prediction successfully sent to Firebase
Fetching new data...
Data received: {'AccelX': '-1.51', 'AccelY': '0.12', 'AccelZ': '9.03', 'GyroX': '-3.00', 'GyroY': '2.75', 'GyroZ': '1.96'}
Prediction made: Fall (Value: 1)
Prediction successfully sent to Firebase
Fetching new data...
Data received: {'AccelX': '-1.51', 'AccelY': '0.12', 'AccelZ': '9.03', 'GyroX': '-3.00', 'GyroY': '2.75', 'GyroZ': '1.96'}
Prediction made: Fall (Value: 1)
Prediction successfully sent to Firebase
Fetching new data...
Data received: {'AccelX': '-1.51', 'AccelY': '0.12', 'AccelZ': '9.03', 'GyroX': '-3.00', 'GyroY': '2.75', '